# Selenium 네이버 뉴스 댓글 스크래핑

## Selenium 및 웹 드라이버 설치

In [2]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromiun-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromiun-browser/chromedriver')

     |████████████████████████████████| 911kB 2.8MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.

## 라이브러리 import

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

## `news_scraping()`: 뉴스 스크래핑

In [9]:
def news_scraping(news_url, wd):

    press = wd.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[1]/a/img').get_attribute("title")
    title = wd.find_element_by_id('articleTitle').text
    datetime = wd.find_element_by_class_name('t11').text
    article = wd.find_element_by_id('articleBodyContents').text
    article = article.replace("// flash 오류를 우회하기 위한 함수 추가","")
    article = article.replace("function _flash_removeCallback() {}","")
    article = article.replace("\n", "")
    article = article.replace("\t", "")

    good = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[1]/a/span[2]').text
    warm = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[2]/a/span[2]').text
    sad  = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[3]/a/span[2]').text
    angry = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[4]/a/span[2]').text
    want = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[5]/a/span[2]').text
    recom = wd.find_element_by_xpath('//*[@id="toMainContainer"]/a/em[2]').text

    print("뉴스:", [title, press, datetime, article, good, warm, sad, angry, want, recom, news_url])
    
    return [title, press, datetime, article, good, warm, sad, angry, want, recom, news_url]



## `comments_scraping()`: 뉴스 댓글 스크래핑

In [10]:
def comments_scraping(news_url, wd):
    try:
        wd.find_element_by_class_name("u_cbox_btn_view_comment").click()
        print("[댓글 더보기]", end="")
        time.sleep(1)
        
        while True:
            wd.find_element_by_class_name("u_cbox_btn_more").click()
            print("[댓글 더보기]", end="")
            time.sleep(1)

    except:
        pass

    btn_reply_list = wd.find_elements_by_class_name("u_cbox_btn_reply")
    for btn_reply in btn_reply_list:
        btn_reply.send_keys('\n')
        print("[답글]", end="")
        time.sleep(1)

    print("[댓글 스크래핑]")
    comments_idx = 0
    contents_df = pd.DataFrame(columns = ["Contents", "Name", "Datetime", "Recommend", "Unrecommend", "URL"])

    comments = wd.find_elements_by_class_name("u_cbox_comment_box")
    for comment in comments:
        try:

            name = comment.find_element_by_class_name("u_cbox_nick").text
            date = comment.find_element_by_class_name("u_cbox_date").text
            contents = comment.find_element_by_class_name("u_cbox_contents").text
            recomm = comment.find_element_by_class_name("u_cbox_cnt_recomm").text
            unrecomm = comment.find_element_by_class_name("u_cbox_cnt_unrecomm").text
            print(f" 댓글 #{comments_idx+1}:", [contents, name, date, recomm, unrecomm, news_url])
            comments_df.loc[comments_idx] = [contents, name, date, recomm, unrecomm, news_url]
            comments_idx += 1
        except NoSuchElementException:
            print("[삭제되거나 부적절한 댓글]")
            continue

    return comments_df

## `scraping()`: 스크래핑 함수

In [11]:
def scraping(news_max = 3):
    wd = webdriver.Chrome("chromedriver", options = chrome_options)
    wd.implicitly_wait(3) 

    wd.execute_script('window.open("about:blank", "_blank");')
    tabs = wd.window_handles

    wd.switch_to.window(tabs[0])
    query = input("검색어 입력: ")
    search_url = "https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=" + query
    wd.get(search_url)


    news_idx = 0
    news_df = pd.DataFrame(columns = ("Title", 'Press', 'Datetime', 'Article', 'Good',
                                      'Warm', 'Sad', 'Angry', 'Want', 'Recommend', 'URL'))
    comments_df = pd.DataFrame()

    while True:
        inline_list = wd.find_elements_by_class_name("txt_inline")
        for inline in inline_list:
            try:
                news_url = inline.find_element_by_class_name("_sp_each_url").get_attribute("href")
            except:
                continue

            wd.switch_to.window(tabs[1])    
            wd.get(news_url)

            news_df.loc[news_idx] = news_scraping(news_url, wd)
            news_idx += 1

            df = comments_scraping(news_url, wd)
            comments_df = pd.concat([comments_df, df])

            if news_idx >= news_max:
                break
        if news_idx >= news_max:
            break
        try:
            wd.swith_to.window(tabs[0])
            wd.find_element_by_class_name("next").click()
            time.sleep(1)
        except:
            break
    
    wd.close()

    return news_df, comments_df

    

## 스크래핑 실행

In [13]:
news_df, comments_df = scraping()


뉴스: ['진중권 "통합당, 광화문집회와 거리둬야…文·여당 지지율 오를라"', '조선일보', '2020.08.15. 오후 7:02', '진 "코로나사태 한복판에 대형집회? 동료시민에 배려없어"중도층 "그런 모습 혐오스러워서 야당 지지 못해"보수 단체 집회 참가자들이 15일 오전 서울 종로구 광화문광장에서 8·15 국민대회 집회중 경찰이 세워놓은 통제선을 넘어 거리로 나오고 있다.마스크를 벗고 경찰에 항의하는 참가자(가운데 빨간 원)도 보인다./뉴시스진중권 전 동양대 교수는 15일 “통합당이 광화문 집회에 선을 긋는 게 중요하다”며 “(이러다간) 다음주에 대통령과 민주당 지지율이 오를 것”이라고 했다.정부는 이날 수도권 지역 코로나가 최대 규모로 확산하는 데 대해 “절체절명 위기”라며 사회적 거리두기를 2단계로 격상한다고 밝혔다. 그럼에도 일부 보수단체 등에 소속된 수만 명이 광화문 집회에 참석했다.130여명 교인이 코로나 확진 판정을 받은 사랑제일교회의 전광훈 대표목사는 보건 당국으로부터 ‘자가 격리’ 통보를 받았음에도 집회 연단에 올랐다. 전 목사는 이날 광화문에서 열린 ‘문재인 퇴진 8·15 범국민대회’에 참석, “오늘도 저를 이 자리에 못 나오게 하려고 바이러스 균을 우리 교회에 부어버렸다”고 주장했다.진 전 교수는 “종교적·정치적 광신에 빠진 사람들은 어는 나라에나, 어느 진영에나 있기 마련”이라며 “그들을 주변화하는 것이 중요하다”고 했다. 진 전 교수는 “조국 사태 때 광화문에 수십만이 모였어도, 별 볼 일 없었죠? 어차피 그 집회에 확장성이 없었기 때문”이라며 “(일반 시민들이) 아무리 정권에 비판적이더라도 태극기 집회에 몸을 보탤 수는 없다고 느끼거든요”라고 했다.진중권 전 동양대 교수./조선일보DB이어 “코로나 사태 한복판에 저런 대형 집회를 연다는 것은 저들의 머릿속에 정치적·종교적 광신만 있을 뿐, 동료 시민에 대한 배려, 더불어 살아가겠다는 의지 따위는 전혀 들어 있지 않다는 것을 보여준다”며 “과거에는 저게 보수의 일반적, 전형적 모습이

KeyboardInterrupt: ignored

In [ ]:
news_df

In [ ]:
comments_df